In [1]:
import pandas as pd
import scipy
import pyodbc

import time

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-IFK6D8L3\\SQLEXPRESS;DATABASE=StockQuote;UID=sa;PWD=password')
cur = conn.cursor()


SQL = """ SELECT        Script_Name, trnx, cls,round((cls*(2/(9+1)))+( MA9 * (1-(2/(9+1)))),2) EMA9,round(MA9, 2)MA
FROM            (SELECT        ne.Script_Name Script_Name, ne.Trnx_date trnx, ne.[Close] cls, avg(ne.[Close])
OVER (partition BY ne.Script_Name ORDER BY ne.Trnx_date rows 9 preceding) AS MA9
FROM            NSE_EOD ne) AS t where  t.Script_Name='HDFCBank' order by trnx
"""

df = pd.read_sql(SQL,conn)
display(df)

,Script_Name,trnx,cls,EMA9Krishna,MA
0,HDFCBANK,2021-11-08,1600.25,1600.25,1600.25
1,HDFCBANK,2021-11-09,1572.25,1586.25,1586.25
2,HDFCBANK,2021-11-10,1555.25,1575.92,1575.92
3,HDFCBANK,2021-11-11,1548.30,1569.01,1569.01
4,HDFCBANK,2021-11-12,1553.00,1565.81,1565.81
...,...,...,...,...,...
275,HDFCBANK,2022-12-19,1644.75,1634.27,1634.27
276,HDFCBANK,2022-12-20,1633.40,1636.49,1636.49
277,HDFCBANK,2022-12-21,1617.60,1637.20,1637.20
278,HDFCBANK,2022-12-22,1612.05,1636.46,1636.46


In [26]:
for i in [9,12,21,25,26,50,100]:
    print(i)
    df['ema'+str(i)] = df['cls'].ewm(span=i).mean()

df

9
12
21
25
26
50
100


,Script_Name,trnx,cls,EMA9Krishna,MA,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema9new
0,HDFCBANK,2021-11-08,1600.25,1600.25,1600.25,1600.250000,1600.250000,1600.250000,1600.250000,1600.250000,1600.250000,1600.250000,1600.250000
1,HDFCBANK,2021-11-09,1572.25,1586.25,1586.25,1584.694444,1585.083333,1585.583333,1585.690000,1585.711538,1585.970000,1586.110000,1586.110000
2,HDFCBANK,2021-11-10,1555.25,1575.92,1575.92,1572.627049,1573.439376,1574.494713,1574.721215,1574.767003,1575.317724,1575.616921,1575.616921
3,HDFCBANK,2021-11-11,1548.30,1569.01,1569.01,1564.386179,1565.503858,1566.982288,1567.303016,1567.367986,1568.152734,1568.581462,1568.581462
4,HDFCBANK,2021-11-12,1553.00,1565.81,1565.81,1560.999048,1562.106604,1563.629113,1563.967191,1564.035991,1564.874995,1565.339284,1565.339284
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,HDFCBANK,2022-12-19,1644.75,1634.27,1634.27,1636.478445,1632.186333,1616.839414,1609.413250,1607.558815,1568.371029,1519.285960,1519.285960
276,HDFCBANK,2022-12-20,1633.40,1636.49,1636.49,1635.862756,1632.373051,1618.344922,1611.258385,1609.472976,1570.921224,1521.554550,1521.554550
277,HDFCBANK,2022-12-21,1617.60,1637.20,1637.20,1632.210205,1630.100274,1618.277202,1611.746201,1610.074978,1572.751791,1523.463787,1523.463787
278,HDFCBANK,2022-12-22,1612.05,1636.46,1636.46,1628.178164,1627.323309,1617.711093,1611.769570,1610.221276,1574.292920,1525.224611,1525.224611


In [44]:
# df['ema100rev'] = df['cls'][::-1].ewm(span=100).mean()
df = df.sort_values(by=['trnx'])
df['ema100new'] = df['cls'].ewm(span=100, adjust=False).mean()

In [20]:
df

,Script_Name,trnx,cls,EMA9Krishna,MA,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema9new
0,HDFCBANK,2021-11-08,1600.25,1600.25,1600.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1600.250000
1,HDFCBANK,2021-11-09,1572.25,1586.25,1586.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1584.694444
2,HDFCBANK,2021-11-10,1555.25,1575.92,1575.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1572.627049
3,HDFCBANK,2021-11-11,1548.30,1569.01,1569.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1564.386179
4,HDFCBANK,2021-11-12,1553.00,1565.81,1565.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1560.999048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,HDFCBANK,2022-12-19,1644.75,1634.27,1634.27,1636.478445,1632.186333,1616.839414,1609.413250,1607.558815,1568.371540,1519.610230,1636.478445
276,HDFCBANK,2022-12-20,1633.40,1636.49,1636.49,1635.862756,1632.373051,1618.344922,1611.258385,1609.472976,1570.921676,1521.863493,1635.862756
277,HDFCBANK,2022-12-21,1617.60,1637.20,1637.20,1632.210205,1630.100274,1618.277202,1611.746201,1610.074978,1572.752198,1523.759265,1632.210205
278,HDFCBANK,2022-12-22,1612.05,1636.46,1636.46,1628.178164,1627.323309,1617.711093,1611.769570,1610.221276,1574.293288,1525.507597,1628.178164


In [45]:
import datetime
df.loc[df['trnx'] == datetime.date(year=2022,month=11,day=11)]

,Script_Name,trnx,cls,EMA9Krishna,MA,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema9new,ema100pta,ema9pta,ema12pta,ema21pta,ema100rev,ema100new
249,HDFCBANK,2022-11-11,1611.15,1514.06,1514.06,1522.738751,1511.730141,1491.552103,1486.239836,1485.114554,1467.696867,1448.978834,1618.031835,1449.984303,1522.738751,1511.730141,1491.552103,1618.031835,1449.997921


In [33]:
import pandas_ta as ta

In [36]:
df['ema100pta'] = ta.ema(df["cls"], length=100)
df['ema9pta'] = ta.ema(df["cls"], length=9)
df['ema12pta'] = ta.ema(df["cls"], length=12)
df['ema21pta'] = ta.ema(df["cls"], length=21)

In [37]:
df.loc[df['trnx'] == datetime.date(year=2022,month=11,day=11)]

,Script_Name,trnx,cls,EMA9Krishna,MA,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema9new,ema100pta,ema9pta,ema12pta,ema21pta
249,HDFCBANK,2022-11-11,1611.15,1514.06,1514.06,1522.738751,1511.730141,1491.552103,1486.239836,1485.114554,1467.696867,1448.978834,1618.031835,1449.984303,1522.738751,1511.730141,1491.552103


In [4]:
df.to_csv("./test2.csv")